In [2]:
!pip install bertopic
!pip install cudf-cu11 dask-cudf-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
!pip install cuml-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
!pip install cugraph-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
!pip uninstall cupy-cuda115 -y
!pip uninstall cupy-cuda11x -y
!pip install cupy-cuda11x -f https://pip.cupy.dev/aarch64

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 53.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
    

In [3]:
from bertopic import BERTopic

In [4]:
from google.colab import files
uploaded = files.upload()

Saving Sentiment_Analysis_British_Airways.csv to Sentiment_Analysis_British_Airways.csv


In [13]:
import pandas as pd
datax=pd.read_csv('Sentiment_Analysis_British_Airways.csv')

In [15]:
import re
import string

In [16]:
# Convert to list
data = datax.comments.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]


In [17]:
import gensim
from gensim import corpora

In [19]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [20]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['regarding', 'the', 'aircraft', 'and', 'seat', 'the', 'business', 'class', 'seats', 'on', 'this', 'flight', 'were', 'pretty', 'good', 'all', 'were', 'of', 'the', 'suite', 'type', 'where', 'every', 'seat', 'has', 'its', 'own', 'booth', 'with', 'sliding', 'door', 'for', 'extra', 'privacy', 'the', 'downside', 'to', 'this', 'is', 'that', 'if', 'you', 'are', 'travelling', 'as', 'couple', 'its', 'quite', 'difficult', 'to', 'interact', 'there', 'is', 'ample', 'storage', 'and', 'usb', 'charging', 'ports', 'the', 'seat', 'is', 'pretty', 'comfortable', 'as', 'someone', 'who', 'finds', 'it', 'very', 'difficult', 'to', 'sleep', 'on', 'flights', 'this', 'was', 'one', 'of', 'the', 'only', 'times', 'ive', 'been', 'able', 'to', 'get', 'few', 'minutes', 'sleep', 'here', 'and', 'there', 'the', 'amenities', 'provided', 'are', 'pretty', 'average', 'when', 'compared', 'to', 'other', 'airlines', 'the', 'leather', 'toiletry', 'bag', 'was', 'quite', 'nice', 'though', 'no', 'slippers', 'only', 'pair', 'of', '

In [21]:
# NLTK Stop words
import nltk
nltk.download('stopwords') # run this one time
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [22]:
from gensim.utils import simple_preprocess

In [23]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [24]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [28]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
!python3 -m spacy download en
!python -m spacy download en_core_web_md
import spacy

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

2023-05-03 06:33:44.755725: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 53.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
2023-05-03 06:33:59.742125: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_md-3.5.0/en_core_web_md-3.5.0-py3-none-any.whl (42.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
[['

In [6]:
topic_model=BERTopic(embedding_model='all-MiniLM-L6-v2')

In [44]:
data_fin=[]
for x in data_lemmatized:
  doc=[]
  for word in x:
    doc.append(word) 
  sent=' '.join(doc)
  data_fin.append(sent)

print(data_fin[1])

travel los_angeles arrive lax bag miss information whereabouts bag search minute find point contact tell bag leave behind give detail stay tell contact day still contact try call customer service give file reference number quite difficult try locate bag finally find reference number bag say deliver couple day tell switch hotel give address stay fine couple week go bag still nowhere see time fly home airport try talk personnel try say deliver first hotel stay even give update address ask fly home guy counter tell problem figure question response walked_away manager see come apologize colleague send fetch bag send flight next come home couple day later receive bag make complaint claim website buy clothe toiletry vacation say refund claim almost still receive refund even multiple email conversation know anymore situation appear willing help appal situation handle frankly disappoint british lack customer service


In [45]:
topics, probs = topic_model.fit_transform(data_fin)

In [46]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,1910,-1_flight_seat_service_fly
1,0,171,0_seat_flight_get_service
2,1,153,1_delay_hour_flight_miss
3,2,124,2_good_comfortable_flight_friendly
4,3,106,3_lounge_club_europe_good_excellent
5,4,99,4_cancel_refund_voucher_book
6,5,84,5_airline_fly_ba_staff
7,6,81,6_class_business_seat_first
8,7,65,7_crew_seat_good_cabin
9,8,59,8_economy_premium_fly_airline


In [47]:
topic_model.get_topic(0)

[('seat', 0.015856364180018335),
 ('flight', 0.01580750955642061),
 ('get', 0.013738358758721038),
 ('service', 0.01306126720023592),
 ('take', 0.012298104786804232),
 ('crew', 0.011660902733467463),
 ('passenger', 0.011505855128312058),
 ('first', 0.011363399271210878),
 ('class', 0.011330019790133055),
 ('go', 0.011277480913510052)]

In [49]:
topic_model.get_representative_docs(0)

['mix class first class check friendly staff fast priority security lane first lounge min unfortunately first lounge full member sit lounge good menu eat grind decent selection wine beer fast boarding start greet door take excellent rose champagne serve friendly professional crew weird tasty cure dry salmon follow steak well cook good cheese course select available decent sleep fast breakfast serve late possible end decent flight north lounge pack min wait shower look tatty dirty decent breakfast selection hot roll fruit cereal leg_room inch cramp like table middle several couple ask remove ok snack cold meat stingy portion time arrival fast bag finish good cramp outward leg return economy possibly bad short_haul flight ever include full cramp insufficient bin service infamous terrible leg_room understand get even bad ba move pax line charter carrier slow food drink service little take buy cash accept even juice water coffee offer free penny_pinching deny even glass water make canadian

In [50]:
df=pd.DataFrame({"topic":topics,"document":data_fin})
df

,topic,document
0,0,regard aircraft seat business class seat fligh...
1,10,travel los_angeles arrive lax bag miss informa...
2,-1,food lousy ever plan asian clueless meal inclu...
3,23,bad experience get delay miss connect flight h...
4,-1,ground staff helpful feel want rush check pass...
...,...,...
3532,-1,fly operate aircraft aircraft clearly new clea...
3533,2,address club passenger name boarding little la...
3534,-1,work urge fly recommend join club go ahead boo...
3535,2,really nice seat professional crew meal serve ...


In [52]:
topic_model.visualize_topics()

In [53]:
topic_model.visualize_barchart()

In [54]:
topic_model.visualize_hierarchy()


In [56]:
topic_model.visualize_heatmap()

In [57]:
topic_model.visualize_term_rank()


In [61]:
topic_model.visualize_term_rank(log_scale=True)